In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pyfusion.algs import eed
from pyfusion.data import load
from pyfusion.structs import math
from pyfusion.structs.validation_utils import FloatArr
from pyfusion.utils import projection, operations

In [ ]:
#type: ignore
def plot(data, title=None):
    """
    Plot the data with optional title and axis labels.
    """
    plt.figure(figsize=(10, 8))
    plt.imshow(data, cmap="gray")
    if title:
        plt.title(title)
    plt.axis("off")
    plt.colorbar()
    plt.show()

In [ ]:
SLICE = 70

In [ ]:
MAX_ITER = 10
STEP_SIZE = 0.2
SIGMA = 2.0

In [ ]:
FSL = load.load_fsl("../data/example_mri", SLICE)

In [ ]:
# type: ignore
# Plotting the FA map
plot(FSL.FA, title="FA map")

In [ ]:
# add random noise to the FA map
FA_noisy = FSL.FA + 0.3 * FSL.FA.std() * np.random.randn(*FSL.FA.shape)

In [ ]:
# type: ignore
# Plotting the noisy FA map
plot(FA_noisy, title="Noisy FA map")

In [ ]:
def heat_conduction(x: FloatArr) -> FloatArr:
    """
    Heat conduction function for the diffusion tensor.
    """
    # copy the input array to avoid modifying the original
    x = x.copy()

    C_4 = 3.31488

    # save the indices of all values <= 0
    neg_indices = x <= 0
    pos_indices = ~neg_indices

    # set the values at those indices to 1
    x[neg_indices] = 1

    # set the values of all other indices to 1 - exp(-C_4 / x[pos_indices])
    x[pos_indices] = 1 - np.exp(-C_4 / x[pos_indices])

    return x

In [ ]:
img = math.Image(data=FA_noisy.copy())

smoothed_img = operations.smooth(img, sigma=SIGMA)

structure_tensort = operations.structure_tensor(smoothed_img)

decomposed_tensor = projection.calculate_decomposition(structure_tensort)

heated_tensor = eed.apply_heat_conduction(decomposed_tensor, heat_conduction)

In [ ]:
for _ in range(MAX_ITER):
    img = eed.step(img, heated_tensor, step_size=STEP_SIZE, sigma=SIGMA)

In [ ]:
plot(
    img.data,
    title="Diffusion Tensor Image after Heat Conduction and EED",
)

In [ ]:
plot(FSL.FA - img.data, title="Difference between Original and EED Image")